In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
import scipy as sp
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from fractions import Fraction
from decimal import Decimal
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 16
refRatio = 2

deriv = 'DD'
order = 1
face = 'R'

L = 1.
locs = [0.5]
epsilons = [1, 0.5]# 11.68]
mus = [1, 0.5]#0.99837]

In [3]:
omegaAMR = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)))# + int(nh / refRatio))
omegaAMR.AddPatch(refRatio, finehalf)
degFreed = omegaAMR.degFreed
nh_max = omegaAMR.nh_max

omegaF = BT.Grid(nh_max)
omegaC = BT.Grid(nh_min)

wavesAMR = WT.MakeWaves(omegaAMR)
nullspace = OT.FindNullspace(omegaAMR, wavesAMR)
nullspaceBlock = OT.Block(nullspace, var = 2)
restrictOp = GTT.CoarsenOp(omegaAMR)
restrictOpBlock = OT.Block(restrictOp, var = 2)

wavesF = WT.MakeWaves(omegaF)
wavesC = WT.MakeWaves(omegaC)

In [4]:
physicsAMR = BT.PhysProps(omegaAMR, epsilons, mus, locs, L)
c = physicsAMR.cVec
cMatAMR = physicsAMR.cMat

physicsC = BT.PhysProps(omegaC, epsilons, mus, locs, L)
cC = physicsC.cVec
cMatC = physicsC.cMat

physicsF = BT.PhysProps(omegaF, epsilons, mus, locs, L)
cF = physicsF.cVec
cMatF = physicsF.cMat

cVecC = physicsC.cVec
c1 = cVecC[0]
c2 = cVecC[-1]

In [5]:
def UDStencil(orderIn):
    errorLoc = 'ERROR:\nOperatorTools:\nUDStencil:\n'
    errorMess = ''
    if (orderIn % 2 == 0):
        order = int(orderIn + 1)
    else:
        order = orderIn
    
    stenc = np.zeros(order + 1)
    
    if (order == 1):
        face = np.asarray([1])
    else:
        if (order == 3):
            face = (1. / 6.) * np.asarray([-1, 5, 2])
        else:
            if (order == 5):
                face = (1. / 60.) * np.asarray([2, -13, 47, 27, -3])
            else:
                if (order == 7):
                    face = (1. / 420.) * np.asarray([-3, 25, -101, 319, 214, -38, 4])
                else:
                    if (order == 9):
                        face = (1. / 2520.) * np.asarray([4, -41, 199, -641, 1879, 1375, -305, 55, -5])
                    else:
                        errorMess = 'This program is not designed to handle this order of accuracy for forward- and backward-difference operators.'
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    return face

In [6]:
def DDStencil(order):
    stenc = UDStencil(order)[::-1]
    return stenc

In [7]:
def FaceOp(omega, physics, order, face, deriv):
    errorMess = ''
    
    # Unpack requisite attributes.
    degFreed = omega.degFreed
    hs = omega.h
    matInd = physics.matInd + 1
    
    # Initialize faceOp.
    faceOp = np.zeros((degFreed, degFreed), float)
    
    # Find patch boundaries.
    spots = np.roll(hs, 1) - hs
    if (all(spots == 0)):
        ps = []
        qs = []
    else:
        # Index after fine-coarse interface
        ps = np.where(spots < 0)[0]
        # Index after coarse-fine interface
        qs = np.where(spots > 0)[0]
        print('ps:')
        print(ps)
        print('qs:')
        print(qs)
        print('')
    
    # Instructions for face approximations.
    if ((deriv == 'U') or (deriv == 'UD')):
        stenc = UDStencil(order)
        if (order % 2 == 0):
            sizeStenc = order + 1
        else:
            sizeStenc = order
        if (face == 'R'):
            off = int((sizeStenc - 1) / 2)
        else:
            if (face == 'L'):
                off = int((sizeStenc + 1) / 2)
    else:
        if ((deriv == 'D') or (deriv == 'DD')):
            stenc = DDStencil(order)
            if (order % 2 == 0):
                sizeStenc = order + 1
            else:
                sizeStenc = order
            if (face == 'R'):
                off = int((sizeStenc - 3) / 2)
            else:
                if (face == 'L'):
                    off = int((sizeStenc - 1) / 2)
        else:
            errorMess = 'This function is not yet set up for anything but upwind differences!'
    
    # Error check.
    if(sizeStenc != len(stenc)):
        errorMess = 'You fucked up the stencil size somehow.'
    if (errorMess != ''):
        sys.exit(errorMess)
    
    # Fill faceOp.
    for i in range(sizeStenc):
        k = int(off - i)
        
        # Create raw face matrix for one coefficient.
        faceMat = np.zeros((degFreed, degFreed), float)
        np.fill_diagonal(faceMat, stenc[i])
        faceMat = np.roll(faceMat, k, axis = 0)
        
        # Create operator for polynomial interpolation.
        polyOp = np.eye(degFreed)
#         if (k > 0):
#             for p in ps:
#                 j = int(off - k)
#                 pAt = p
#                 pLow = (p - 1) % degFreed
#                 pHi = (p + 1) % degFreed
#                 for g in range(k):
#                     polyMat[pAt, :] = 0
#                     polyMat[pAt, pLow:pHi] = 0.5
#                     pAt = (pAt - 1) % degFreed
#                     pLow = (pLow - 2) % degFreed
#                     pHi = (pHi - 2) % degFreed
#                     j = int(j + 1)
#             for q in qs:
#                 j = int(off - k)
#                 qAt = (q - k + 1) % degFreed
#                 for g in range(k):
#                     polyMat[qAt, :] = polyStencSet[j, :]
#                     qAt = (qAt + 1) % degFreed
#                     j = int(j + 1)
        
        
        
        faceMat = polyOp @ faceMat
        
        print('A^(' + str(k) + ') =')
        print(polyOp)
        print('')
        print('D^(' + str(k) + ') =')
        print(faceMat)
        print('')
        faceOp = faceOp + faceMat
    
    
    hMat = OT.StepMatrix(omega)
    faceOp = hMat @ faceOp
    
    return faceOp

In [8]:
faceOp = FaceOp(omegaAMR, physicsAMR, order, face, deriv)
print('D =')
print(faceOp)
print('')

ps:
[16]
qs:
[0]

A^(-1) =
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [9]:
stenc1 = UDStencil(order)
stenc2 = DDStencil(order)
print(stenc1)
print(stenc2)
print('')

[1]
[1]

